# 📊 Database Table Copy Notebook

This notebook copies tables from a source SQLite database to the intake-crm.db database.

**Features:**
- Interactive table selection
- Multiple copy modes (replace, append, skip duplicates)
- Schema inspection and creation
- Progress tracking and validation

## 📁 Configuration

Set your database paths and preferences here:

In [4]:
import sqlite3
import pandas as pd
from pathlib import Path
from typing import List, Tuple, Dict, Any

# Configuration - Update these paths
SOURCE_DB = r"C:\Users\ChristopherCato\OneDrive - clarity-dx.com\code\monolith\monolith.db"  # Change this to your source database
DEST_DB = r"C:\Users\ChristopherCato\OneDrive - clarity-dx.com\code\intake-crm\intake-crm.db"  # Your intake CRM database

# Tables you want to copy (update this list)
TABLES_TO_COPY = ["providers", "ppo"]  # Change these to your actual table names

# Copy mode: 'replace', 'append', or 'skip_existing'
COPY_MODE = "replace"

print(f"📁 Source Database: {SOURCE_DB}")
print(f"📁 Destination Database: {DEST_DB}")
print(f"📋 Tables to Copy: {TABLES_TO_COPY}")
print(f"🔄 Copy Mode: {COPY_MODE}")

📁 Source Database: C:\Users\ChristopherCato\OneDrive - clarity-dx.com\code\monolith\monolith.db
📁 Destination Database: C:\Users\ChristopherCato\OneDrive - clarity-dx.com\code\intake-crm\intake-crm.db
📋 Tables to Copy: ['providers', 'ppo']
🔄 Copy Mode: replace


## 🔍 Database Inspection

Let's first examine what tables are available in the source database:

In [5]:
def list_tables_with_info(db_path: str) -> pd.DataFrame:
    """Get a DataFrame with table information."""
    if not Path(db_path).exists():
        print(f"❌ Database not found: {db_path}")
        return pd.DataFrame()
    
    conn = sqlite3.connect(db_path)
    
    # Get table names
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name")
    tables = [row[0] for row in cursor.fetchall()]
    
    # Get info for each table
    table_info = []
    for table in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table}")
        row_count = cursor.fetchone()[0]
        
        cursor.execute(f"PRAGMA table_info({table})")
        columns = cursor.fetchall()
        column_count = len(columns)
        
        table_info.append({
            'Table Name': table,
            'Row Count': row_count,
            'Column Count': column_count
        })
    
    conn.close()
    return pd.DataFrame(table_info)

# Show source database tables
print("📋 Tables in Source Database:")
source_tables = list_tables_with_info(SOURCE_DB)
display(source_tables)

📋 Tables in Source Database:


,Table Name,Row Count,Column Count
0,AdjustmentReason,0,4
1,BillLineItem,2643,12
2,EOBR,0,6
3,FeeSchedule,0,6
4,ProviderBill,1390,19
5,ReimbursementLog,0,5
6,assignment_email,0,11
7,assignment_email_attachments,0,7
8,assignment_email_category,0,2
9,auth_group,0,2


In [6]:
# Show destination database tables (if it exists)
print("📋 Tables in Destination Database:")
dest_tables = list_tables_with_info(DEST_DB)
if not dest_tables.empty:
    display(dest_tables)
else:
    print("🆕 Destination database is empty or doesn't exist")

📋 Tables in Destination Database:


,Table Name,Row Count,Column Count
0,email_metadata,4,13
1,referrals,4,39
2,sqlite_sequence,2,2


## 🔧 Helper Functions

Define functions for table operations:

In [7]:
def get_table_schema(conn: sqlite3.Connection, table_name: str) -> str:
    """Get the CREATE TABLE statement for a table."""
    cursor = conn.cursor()
    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        raise ValueError(f"Table '{table_name}' not found")

def table_exists(conn: sqlite3.Connection, table_name: str) -> bool:
    """Check if a table exists."""
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
    return cursor.fetchone() is not None

def get_table_sample(db_path: str, table_name: str, limit: int = 5) -> pd.DataFrame:
    """Get a sample of data from a table."""
    conn = sqlite3.connect(db_path)
    try:
        df = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT {limit}", conn)
        return df
    finally:
        conn.close()

def copy_table_data(source_conn: sqlite3.Connection, dest_conn: sqlite3.Connection, 
                   table_name: str, mode: str = "replace") -> Dict[str, int]:
    """Copy data from source table to destination table."""
    source_cursor = source_conn.cursor()
    dest_cursor = dest_conn.cursor()
    
    # Get all data from source table
    source_cursor.execute(f"SELECT * FROM {table_name}")
    rows = source_cursor.fetchall()
    
    if not rows:
        return {"copied": 0, "skipped": 0, "total": 0}
    
    # Get column names
    source_cursor.execute(f"PRAGMA table_info({table_name})")
    columns = [col[1] for col in source_cursor.fetchall()]
    column_list = ", ".join(columns)
    placeholders = ", ".join(["?" for _ in columns])
    
    rows_copied = 0
    rows_skipped = 0
    
    if mode == "replace":
        # Clear destination table first
        dest_cursor.execute(f"DELETE FROM {table_name}")
        dest_cursor.executemany(f"INSERT INTO {table_name} ({column_list}) VALUES ({placeholders})", rows)
        rows_copied = len(rows)
        
    elif mode == "append":
        # Try to insert all rows
        for row in rows:
            try:
                dest_cursor.execute(f"INSERT INTO {table_name} ({column_list}) VALUES ({placeholders})", row)
                rows_copied += 1
            except sqlite3.IntegrityError:
                rows_skipped += 1
    
    elif mode == "skip_existing":
        # Use INSERT OR IGNORE
        dest_cursor.executemany(f"INSERT OR IGNORE INTO {table_name} ({column_list}) VALUES ({placeholders})", rows)
        rows_copied = dest_cursor.rowcount
        rows_skipped = len(rows) - rows_copied
    
    dest_conn.commit()
    return {"copied": rows_copied, "skipped": rows_skipped, "total": len(rows)}

print("✅ Helper functions defined")

✅ Helper functions defined


## 👀 Preview Source Data

Let's look at sample data from the tables we want to copy:

In [8]:
# Preview data from each table
for table_name in TABLES_TO_COPY:
    print(f"\n📋 Sample data from '{table_name}':")
    try:
        sample_df = get_table_sample(SOURCE_DB, table_name, limit=3)
        if not sample_df.empty:
            display(sample_df)
        else:
            print("   (Empty table)")
    except Exception as e:
        print(f"   ❌ Error: {e}")


📋 Sample data from 'providers':


,_g_Dashboard_Provider_Diary,_g_Dashboard_Provider_Status,Address 1 Full,Address Line 1,Address Line 2,All,Angiography,Arthrogram,Billing Address 1,Billing Address 2,...,ServicesProvided,State,Status,TIN,US,Website,Wide Bore,xlink,Xray,flag_is_hospital
0,0.0,162.0,"3000 WEST MACARTHUR BLVD STE 600\nSANTA ANA, C...",3000 WEST MACARTHUR BLVD,STE 600,Yes,None,No,PO Box 826618,None,...,- High Field Closed MRI - Xray - MRA - - - ...,CA,Inactive,833563413,Yes,None,None,None,Yes,No
1,0.0,162.0,"2777 JEFFERSON ST STE\nCARLSBAD, CA 92008",2777 JEFFERSON ST,STE,Yes,No,No,None,None,...,- - - - - - - - - - - - - - - ...,CA,Active,833563413,No,None,No,None,No,No
2,0.0,162.0,"9191 WESTMINSTER AVE STE. 209\nGARDEN GROVE, C...",9191 WESTMINSTER AVE,STE. 209,Yes,No,No,None,None,...,- - - - - - - - - - - - - - - ...,CA,Active,833563413,No,None,No,None,No,No



📋 Sample data from 'ppo':


,id,RenderingState,TIN,provider_name,proc_cd,modifier,proc_desc,proc_category,rate
0,1,None,262562595,"A-1 Imaging Centers, LLC",74181,None,MRI ABDOMEN W/O DYE,MRI w/o,285
1,2,None,262562595,"A-1 Imaging Centers, LLC",70551,None,MRI BRAIN STEM W/O DYE,MRI w/o,285
2,3,None,262562595,"A-1 Imaging Centers, LLC",72141,None,MRI NECK SPINE W/O DYE,MRI w/o,285


## 🚀 Execute Table Copy

Now let's copy the tables:

In [14]:
def copy_table_robust(source_db_path: str, dest_db_path: str, table_name: str, mode: str = "replace") -> bool:
    """Copy a single table from source to destination with robust handling."""
    
    print(f"\n📋 Copying table: {table_name}")
    
    # Connect to databases
    source_conn = sqlite3.connect(source_db_path)
    dest_conn = sqlite3.connect(dest_db_path)
    
    try:
        # Check if source table exists
        if not table_exists(source_conn, table_name):
            print(f"❌ Table '{table_name}' not found in source database")
            return False
        
        # Get source table row count
        source_cursor = source_conn.cursor()
        source_cursor.execute(f"SELECT COUNT(*) FROM `{table_name}`")
        source_count = source_cursor.fetchone()[0]
        print(f"   📊 Source rows: {source_count}")
        
        # Check if destination table exists
        if not table_exists(dest_conn, table_name):
            print(f"   🔧 Creating table '{table_name}' in destination...")
            
            # Get column info
            source_cursor.execute(f"PRAGMA table_info(`{table_name}`)")
            columns = source_cursor.fetchall()
            
            # Build a simple CREATE TABLE statement without constraints
            column_defs = []
            for col in columns:
                col_id, col_name, col_type, not_null, default_val, is_pk = col
                
                # Always quote column names to handle spaces and special characters
                safe_col_name = f'"{col_name}"'
                
                # Simplify - just use column name and type, ignore constraints
                col_def = f"{safe_col_name} {col_type}"
                column_defs.append(col_def)
            
            # Create table without any constraints to avoid issues
            create_sql = f'CREATE TABLE "{table_name}" ({", ".join(column_defs)})'
            print(f"   🔧 Creating with simplified schema...")
            
            dest_conn.execute(create_sql)
            dest_conn.commit()
            print(f"   ✅ Table created")
        else:
            dest_cursor = dest_conn.cursor()
            dest_cursor.execute(f"SELECT COUNT(*) FROM `{table_name}`")
            dest_count_before = dest_cursor.fetchone()[0]
            print(f"   📊 Destination rows (before): {dest_count_before}")
        
        # Copy data in batches to handle large tables
        print(f"   🔄 Copying data (mode: {mode})...")
        
        # Get column names and quote them all
        source_cursor.execute(f"PRAGMA table_info(`{table_name}`)")
        columns = source_cursor.fetchall()
        
        quoted_column_names = [f'"{col[1]}"' for col in columns]
        column_list = ", ".join(quoted_column_names)
        placeholders = ", ".join(["?" for _ in quoted_column_names])
        
        dest_cursor = dest_conn.cursor()
        
        if mode == "replace":
            # Clear destination table first
            dest_cursor.execute(f'DELETE FROM "{table_name}"')
            dest_conn.commit()
        
        # Copy data in batches
        batch_size = 1000
        offset = 0
        total_copied = 0
        total_skipped = 0
        
        while True:
            # Get batch of data
            source_cursor.execute(f'SELECT * FROM "{table_name}" LIMIT {batch_size} OFFSET {offset}')
            batch_rows = source_cursor.fetchall()
            
            if not batch_rows:
                break
            
            batch_copied = 0
            batch_skipped = 0
            
            if mode == "replace":
                # Simple insert for replace mode
                dest_cursor.executemany(f'INSERT INTO "{table_name}" ({column_list}) VALUES ({placeholders})', batch_rows)
                batch_copied = len(batch_rows)
                
            elif mode == "append":
                # Insert one by one to handle conflicts
                for row in batch_rows:
                    try:
                        dest_cursor.execute(f'INSERT INTO "{table_name}" ({column_list}) VALUES ({placeholders})', row)
                        batch_copied += 1
                    except sqlite3.IntegrityError:
                        batch_skipped += 1
            
            elif mode == "skip_existing":
                # Use INSERT OR IGNORE
                dest_cursor.executemany(f'INSERT OR IGNORE INTO "{table_name}" ({column_list}) VALUES ({placeholders})', batch_rows)
                batch_copied = dest_cursor.rowcount if dest_cursor.rowcount > 0 else len(batch_rows)
                batch_skipped = len(batch_rows) - batch_copied
            
            total_copied += batch_copied
            total_skipped += batch_skipped
            
            dest_conn.commit()
            
            print(f"   📦 Batch {offset//batch_size + 1}: {batch_copied} copied, {batch_skipped} skipped")
            
            offset += batch_size
        
        # Show final results
        dest_cursor.execute(f'SELECT COUNT(*) FROM "{table_name}"')
        final_count = dest_cursor.fetchone()[0]
        
        print(f"   ✅ Copy complete:")
        print(f"      Total rows copied: {total_copied}")
        print(f"      Total rows skipped: {total_skipped}")
        print(f"      Final count: {final_count}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error copying table '{table_name}': {e}")
        import traceback
        traceback.print_exc()
        return False
    
    finally:
        source_conn.close()
        dest_conn.close()

print("✅ Robust copy function defined")

✅ Robust copy function defined


In [13]:
# Debug the providers table schema
print("🔍 Debugging providers table schema:")

source_conn = sqlite3.connect(SOURCE_DB)
try:
    # Get the original schema
    cursor = source_conn.cursor()
    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table' AND name='providers'")
    schema = cursor.fetchone()[0]
    print("\n📋 Original schema:")
    print(schema)
    
    # Get column info
    cursor.execute("PRAGMA table_info(providers)")
    columns = cursor.fetchall()
    print("\n📋 Column details:")
    for col in columns:
        print(f"   {col[1]} ({col[2]}) - Nullable: {not col[3]} - Default: {col[4]} - PK: {col[5]}")
        
finally:
    source_conn.close()

🔍 Debugging providers table schema:

📋 Original schema:
CREATE TABLE "providers" (
"_g_Dashboard_Provider_Diary" REAL,
  "_g_Dashboard_Provider_Status" REAL,
  "Address 1 Full" TEXT,
  "Address Line 1" TEXT,
  "Address Line 2" TEXT,
  "All" TEXT,
  "Angiography" TEXT,
  "Arthrogram" TEXT,
  "Billing Address 1" TEXT,
  "Billing Address 2" TEXT,
  "Billing Address City" TEXT,
  "Billing Address Postal Code" TEXT,
  "Billing Address State" TEXT,
  "Billing Name" TEXT,
  "Bone Density" TEXT,
  "Breast MRI" TEXT,
  "City" TEXT,
  "Contract Date" TEXT,
  "Contract Date Renewal" TEXT,
  "Country" TEXT,
  "CreatedBy" TEXT,
  "CreationTimestamp" TEXT,
  "CT" TEXT,
  "CT W" TEXT,
  "CT WO" TEXT,
  "CT WWO" TEXT,
  "CTA" TEXT,
  "DBA Name Billing Name" TEXT,
  "Description" TEXT,
  "Dexa" TEXT,
  "Diary Date" TEXT,
  "distance" TEXT,
  "Echo" TEXT,
  "EKG" TEXT,
  "Email" TEXT,
  "EMG" TEXT,
  "Fax Number" TEXT,
  "Fluroscopy" TEXT,
  "ForeignKey" TEXT,
  "g_lat" TEXT,
  "g_lon" TEXT,
  "High Fie

In [16]:
# Retry copying both tables with the robust function
print("🔄 Retrying table copy with robust handling...")

success_count = 0
for table_name in TABLES_TO_COPY:
    if copy_table_robust(SOURCE_DB, DEST_DB, table_name, COPY_MODE):
        success_count += 1

print(f"\n📊 Final Copy Summary:")
print(f"   Total tables: {len(TABLES_TO_COPY)}")
print(f"   Successfully copied: {success_count}")
print(f"   Failed: {len(TABLES_TO_COPY) - success_count}")

if success_count == len(TABLES_TO_COPY):
    print(f"✅ All tables copied successfully!")
else:
    print(f"⚠️  Some tables still failed")

🔄 Retrying table copy with robust handling...

📋 Copying table: providers
   📊 Source rows: 2296
   📊 Destination rows (before): 0
   🔄 Copying data (mode: replace)...
   📦 Batch 1: 1000 copied, 0 skipped
   📦 Batch 2: 1000 copied, 0 skipped
   📦 Batch 3: 296 copied, 0 skipped
   ✅ Copy complete:
      Total rows copied: 2296
      Total rows skipped: 0
      Final count: 2296

📋 Copying table: ppo
   📊 Source rows: 19430
   📊 Destination rows (before): 19430
   🔄 Copying data (mode: replace)...
   📦 Batch 1: 1000 copied, 0 skipped
   📦 Batch 2: 1000 copied, 0 skipped
   📦 Batch 3: 1000 copied, 0 skipped
   📦 Batch 4: 1000 copied, 0 skipped
   📦 Batch 5: 1000 copied, 0 skipped
   📦 Batch 6: 1000 copied, 0 skipped
   📦 Batch 7: 1000 copied, 0 skipped
   📦 Batch 8: 1000 copied, 0 skipped
   📦 Batch 9: 1000 copied, 0 skipped
   📦 Batch 10: 1000 copied, 0 skipped
   📦 Batch 11: 1000 copied, 0 skipped
   📦 Batch 12: 1000 copied, 0 skipped
   📦 Batch 13: 1000 copied, 0 skipped
   📦 Batch 14:

## ✅ Verification

Let's verify the copy was successful:

In [11]:
# Show updated destination database tables
print("📋 Final state of destination database:")
final_dest_tables = list_tables_with_info(DEST_DB)
display(final_dest_tables)

📋 Final state of destination database:


,Table Name,Row Count,Column Count
0,email_metadata,4,13
1,ppo,19430,9
2,providers,0,91
3,referrals,4,39
4,sqlite_sequence,2,2


In [12]:
# Preview the copied data
for table_name in TABLES_TO_COPY:
    print(f"\n📋 Sample data from copied table '{table_name}':")
    try:
        sample_df = get_table_sample(DEST_DB, table_name, limit=3)
        if not sample_df.empty:
            display(sample_df)
        else:
            print("   (Empty table)")
    except Exception as e:
        print(f"   ❌ Error: {e}")


📋 Sample data from copied table 'providers':
   (Empty table)

📋 Sample data from copied table 'ppo':


,id,RenderingState,TIN,provider_name,proc_cd,modifier,proc_desc,proc_category,rate
0,1,None,262562595,"A-1 Imaging Centers, LLC",74181,None,MRI ABDOMEN W/O DYE,MRI w/o,285
1,2,None,262562595,"A-1 Imaging Centers, LLC",70551,None,MRI BRAIN STEM W/O DYE,MRI w/o,285
2,3,None,262562595,"A-1 Imaging Centers, LLC",72141,None,MRI NECK SPINE W/O DYE,MRI w/o,285
